## Data Extraction with Docling

In this notebook, we'll extract content from PDFs into structured formats:

- **Markdown**: Full document text with page breaks for chunking
- **Images**: Save pages containing large charts/diagrams (>500x500 pixels)
- **Tables**: Extract with 2 paragraphs of context + page number metadata

**Pipeline Overview:**
1. **This Notebook (06-01)**: Extract PDFs → Markdown, Images, Tables
2. **Next Notebook (06-02)**: Load into vector database with embeddings
3. **Notebook 07**: Intelligent search with filters and reranking

**Output Structure:**
```
data/markdown/{company}/{document}.md
data/images/{company}/{document}/page_5.png
data/tables/{company}/{document}/table_1_page_5.md
```

### 1. Setup and Configuration

In [ ]:
from pathlib import Path
from typing import List, Tuple

from docling_core.types.doc import PictureItem
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

In [ ]:
# Directory paths
DATA_DIR = "data/rag-data/pdfs"
OUTPUT_MD_DIR = "data/rag-data/markdown"
OUTPUT_IMAGES_DIR = "data/rag-data/images"
OUTPUT_TABLES_DIR = "data/rag-data/tables"

### 2. Helper Functions

In [ ]:
def extract_metadata_from_filename(filename: str) -> dict:
    """
    Extract metadata from filename.
    
    Expected format: CompanyName DocType [Quarter] Year.pdf
    Examples:
        - Amazon 10-Q Q1 2024.pdf
        - Microsoft 10-K 2023.pdf
    """
    parts = filename.split()
    
    return {
        'company_name': parts[0],
        'doc_type': parts[1],
        'fiscal_quarter': parts[2] if len(parts) == 4 else None,
        'fiscal_year': int(parts[-1])
    }

In [ ]:
def extract_context_and_table(lines: List[str], table_index: int) -> Tuple[str, int]:
    """
    Extract context and table content at a specific position.
    
    Args:
        lines: All markdown lines
        table_index: Where the table starts
    
    Returns:
        (combined_content, next_line_index)
    """
    # Get 2 paragraphs before the table
    context_lines = []
    para_count = 0
    j = table_index - 1
    
    while j >= 0 and para_count < 2:
        if lines[j].strip():
            if '<!-- page break -->' not in lines[j]:
                context_lines.insert(0, lines[j])
        elif context_lines:
            para_count += 1
        j -= 1
    
    # Get all table rows
    table_lines = []
    i = table_index
    
    while i < len(lines):
        if lines[i].strip().startswith('|'):
            table_lines.append(lines[i])
            i += 1
        elif not lines[i].strip():
            i += 1  # Skip empty lines within table
        else:
            break  # End of table
    
    # Combine them
    content = '\n'.join(context_lines) + '\n\n' + '\n'.join(table_lines)
    return content, i

In [ ]:
def extract_tables_with_context(markdown_text: str) -> List[Tuple[str, str, int]]:
    """
    Find all tables and extract them with context and page numbers.
    
    Returns:
        List of (content, table_name, page_number)
    """
    lines = markdown_text.split('\n')
    tables = []
    current_page = 1
    table_num = 1
    i = 0
    
    while i < len(lines):
        # Track page numbers
        if '<!-- page break -->' in lines[i]:
            current_page += 1
            i += 1
            continue
        
        # Found a table?
        if lines[i].strip().startswith('|') and lines[i].count('|') >= 2:
            content, next_i = extract_context_and_table(lines, i)
            tables.append((content, f"table_{table_num}", current_page))
            table_num += 1
            i = next_i
        else:
            i += 1
    
    return tables

In [ ]:
def convert_pdf_to_docling(pdf_path: Path):
    """
    Convert PDF using Docling.
    
    Returns:
        Docling conversion result
    """
    pipeline_options = PdfPipelineOptions()
    pipeline_options.generate_picture_images = True
    pipeline_options.generate_page_images = True
    
    converter = DocumentConverter(
        format_options={InputFormat.PDF: PdfFormatOption(pipeline_options=pipeline_options)}
    )
    
    return converter.convert(pdf_path)

In [ ]:
def save_page_images(result, images_dir: Path):
    """
    Find and save pages with large images (>500x500 pixels).
    """
    pages_to_save = set()
    
    for item in result.document.iterate_items():
        element = item[0]
        
        if isinstance(element, PictureItem):
            image = element.get_image(result.document)
            if image.size[0] > 500 and image.size[1] > 500:
                page_no = element.prov[0].page_no if element.prov else None
                if page_no:
                    pages_to_save.add(page_no)
    
    # Save images
    for page_no in pages_to_save:
        page = result.document.pages[page_no]
        page.image.pil_image.save(images_dir / f"page_{page_no}.png", "PNG")
    
    if pages_to_save:
        print(f"  ✓ Saved {len(pages_to_save)} page images")

In [ ]:
def save_tables(markdown_text: str, tables_dir: Path):
    """
    Extract and save tables with context and page numbers.
    """
    tables = extract_tables_with_context(markdown_text)
    
    for table_content, table_name, page_num in tables:
        content_with_page = f"**Page:** {page_num}\n\n{table_content}"
        (tables_dir / f"{table_name}_page_{page_num}.md").write_text(content_with_page, encoding='utf-8')
    
    if tables:
        print(f"  ✓ Saved {len(tables)} tables")

### 3. Main Extraction Function

In [ ]:
def extract_pdf_content(pdf_path: Path):
    """Extract PDF to markdown, images, and tables."""
    print(f"Processing: {pdf_path.name}")
    
    # Setup output directories
    filename = pdf_path.name.replace('.pdf', '')
    metadata = extract_metadata_from_filename(filename)
    company = metadata['company_name']
    filename_stem = pdf_path.stem
    
    md_dir = Path(OUTPUT_MD_DIR) / company
    images_dir = Path(OUTPUT_IMAGES_DIR) / company / filename_stem
    tables_dir = Path(OUTPUT_TABLES_DIR) / company / filename_stem
    
    for dir_path in [md_dir, images_dir, tables_dir]:
        dir_path.mkdir(parents=True, exist_ok=True)
    
    # Convert PDF with Docling
    result = convert_pdf_to_docling(pdf_path)
    
    # Save markdown
    markdown_text = result.document.export_to_markdown(page_break_placeholder="<!-- page break -->")
    (md_dir / f"{filename_stem}.md").write_text(markdown_text, encoding='utf-8')
    print(f"  ✓ Markdown saved")
    
    # Save page images
    save_page_images(result, images_dir)
    
    # Save tables
    save_tables(markdown_text, tables_dir)
    
    print(f"  Done!\n")

### 4. Process a Single PDF (Example)

In [ ]:
# Find all PDF files
data_path = Path(DATA_DIR)
pdf_files = list(data_path.rglob("*.pdf"))
print(f"Found {len(pdf_files)} PDF files\n")

# Process one example first to see the output
if pdf_files:
    print("=== Processing Example PDF ===")
    extract_pdf_content(pdf_files[0])
    print("\nCheck the output folders to see extracted files!")
    print(f"- Markdown: {OUTPUT_MD_DIR}")
    print(f"- Images: {OUTPUT_IMAGES_DIR}")
    print(f"- Tables: {OUTPUT_TABLES_DIR}")

### 5. Process All PDFs

In [ ]:
# Process all PDFs
print(f"\n=== Processing All {len(pdf_files)} PDFs ===\n")

for idx, pdf_path in enumerate(pdf_files, 1):
    print(f"[{idx}/{len(pdf_files)}]", end=" ")
    extract_pdf_content(pdf_path)

print("\n=== Extraction Complete ===")